In [1]:
%load_ext autoreload
%autoreload 2
import optimizer
import json
import pandas as pd
pd.set_option('display.max_rows', 999)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
use_load = pd.read_csv("../Project_data/Predictor Data/load.csv", index_col=0, parse_dates=True)
generation = pd.read_csv("../Project_data/Predictor Data/generation.csv", index_col=0, parse_dates=True)

In [3]:
# Base Assumptions
storage_system = {
    "battery_capacity_kwh": 13.5,
    "efficiency": 0.95,
    "max_discharge_power_kw": 10,
    "max_charge_power_kw": 10
}

def get_rate_and_state(rate_file, use_load):
    with open(rate_file, 'r') as f:
        rate_plan = json.load(f)

    # System state
    system_state = {
        "soc": 0
    }

    rate_list = optimizer.generate_hourly_schedule(use_load.index[0], len(use_load), rate_plan['rate_schedule'])

    for group in rate_list:
        system_state[group] = {
                "from_grid": 0,
                "to_grid": 0
            }
    return rate_plan, system_state

rate_plans = {
    "WE Energies": "../Project_data/Energy_Rates/wi-cgs-nm.json",
    "NEM 2.0": "../Project_data/Energy_Rates/nem2.json",
    "NEM 3.0": "../Project_data/Energy_Rates/nem3.json"
}

battery_capacities = [0, 13.5, 27]

In [4]:
df = pd.DataFrame()
# Predict the month here and reuse for the different other configs
pred_load, pred_gen = optimizer.predict_load_and_gen(use_load, generation)
for month in range(1,13):
    print(month)
    month_pred_load = pred_load[pred_load.index.month == month].copy()
    month_pred_gen = pred_gen[pred_gen.index.month == month].copy()

    for batt_cap in battery_capacities:
        print(f"Battery Capacity: {batt_cap}") # Show Progress
        storage_system['battery_capacity_kwh'] = batt_cap
        print(f"Rate Plan:", end=" ") # Show Progress
        for rp, filename in rate_plans.items():
            print(f"{rp}", end=" ") # Show Progress

            # Run through the different 
            month_load = use_load[use_load['month'] == month].copy()
            month_gen = generation[generation['month'] == month].copy()
            rate_plan, system_state = get_rate_and_state(filename, month_load)
            df.loc[month,f"{rp}, {batt_cap}, No Generation"] = optimizer.no_generation(month_load, rate_plan)
            df.loc[month,f"{rp}, {batt_cap}, Basic Heuristic"] = optimizer.basic_heuristic(month_load, month_gen, rate_plan, system_state, storage_system)[1]
            df.loc[month,f"{rp}, {batt_cap}, Predicted"] = optimizer.monthly_predict_and_optimize(use_load, generation, rate_plan, system_state, storage_system, month, predicted_load=month_pred_load, predicted_generation=month_pred_gen)[1]
            df.loc[month,f"{rp}, {batt_cap}, Stepped Optimized"] = optimizer.monthly_set_and_optimize(use_load, generation, rate_plan, system_state, storage_system, month)[1]
            # This looks at the full month instead of doing a 24 hour look-ahead
            # Also has the issue of not dealing with price inversions seen in NEM 3.0
            # Used for reference only
            recommend_charge, recommend_export, recommend_import, new_system_state, model = optimizer.optimize_period(month_load, month_gen, rate_plan, system_state, storage_system)
            df.loc[month,f"{rp}, {batt_cap}, Fully Optimized"] = model.total_cost()
        print()
df.to_csv('./method_output.csv') # Store the data so we don't have to rerun this expensive cell

1
Battery Capacity: 0
Rate Plan: WE Energies WARNING: DEPRECATED: Param 'rate_group' declared with an implicit domain of
'Any'. The default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend the
domain of this Paramto be 'Any', you can suppress this warning by explicitly
specifying 'within=Any' to the Param constructor.  (deprecated in 5.6.9, will
be removed in (or after) 6.0) (called from
/Users/dandrusch/.pyenv/versions/3.10.6/envs/ds785/lib/python3.10/site-
packages/pyomo/core/base/indexed_component.py:714)
NEM 2.0 NEM 3.0 
Battery Capacity: 13.5
Rate Plan: WE Energies NEM 2.0 NEM 3.0 
Battery Capacity: 27
Rate Plan: WE Energies NEM 2.0 NEM 3.0 
2
Battery Capacity: 0
Rate Plan: WE Energies NEM 2.0 NEM 3.0 
Battery Capacity: 13.5
Rate Plan: WE Energies NEM 2.0 NEM 3.0 
Battery Capacity: 27
Rate Plan: WE Energies NEM 2.0 NEM 3.0 
3
Battery Capacity: 0
Rate Plan: WE Energies NEM 2.0 NEM 3.0 
Battery Capacity: 13

In [5]:
# Sum up the months to get the whole year's cost
summary_df = pd.DataFrame(df.sum())
summary_df.columns = ['Cost'] # Rename column for clarity

In [6]:
# Split out aspects of each simulation to use for reshaping
summary_df[['Rate Plan', 'Battery Capacity', 'Method']] = summary_df.index.to_series().str.split(', ', expand=True)

In [7]:
summary_df[summary_df['Battery Capacity'] == "27"].pivot_table(values='Cost', index='Method', columns='Rate Plan')

Rate Plan,NEM 2.0,NEM 3.0,WE Energies
Method,,,
Basic Heuristic,-1171.968,734.776,19.827
Fully Optimized,-1318.303,-1359.005,-104.122
No Generation,6632.041,6632.041,2480.561
Predicted,-1256.045,-634.581,-65.180
Stepped Optimized,-1316.933,-915.691,-86.229
